In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
USER_ID = "ayoush"
APP_NAME = "testing"
SESSION_ID = "ayoush-session"
MODEL_NAME= "gemini-2.5-flash-lite"

# Helper function to run a chatbot session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n### Session: {session_name}")
    
    # Get or create session
    app_name = runner_instance.app_name
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    
    # Exit if no queries provided
    if not user_queries:
        print("No queries!")
        return
    
    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]
    
    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        
        # Format query for the runner
        formatted_query = types.Content(
            role="user", 
            parts=[types.Part(text=query)]
        )
        
        # Stream and print responses
        async for event in runner_instance.run_async(
            user_id=USER_ID, 
            session_id=session.id, 
            new_message=formatted_query
        ):
            # Print non-empty responses
            if event.content and event.content.parts:
                response_text = event.content.parts[0].text
                if response_text and response_text != "None":
                    print(f"{MODEL_NAME} > {response_text}")

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:

root_agent = Agent(
    model=Gemini(model_name=MODEL_NAME, retry_options=retry_config),
    name="chat_bot_agent",
    description="A helpful assistant that can answer questions and help with tasks."
)

session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

In [ ]:
await run_session(
    runner,
    [
        "Hi, I am Ayoush! What is the capital of United States?",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)

In [ ]:
chatbot_agent = LlmAgent(
    model=Gemini(model_name=MODEL_NAME, retry_options=retry_config),
    name="chatbot_agent",
    description="A helpful assistant that can answer questions and help with tasks."
)
db_url = "sqlite:///ayoush.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")




In [ ]:
await run_session(
    runner,
    ["Hi, I am Sam! What is the capital of the United States?", "Hello! What is my name?"],
    "test-db-session-01",
)

In [ ]:
await run_session(
    runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)

In [ ]:
await run_session(
    runner, ["Hello! What is my name?"], "test-db-session-02"
)  # Note, we are using new session name

In [ ]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

In [ ]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

In [ ]:
# Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)

In [ ]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

In [4]:
def save_user_info_to_db(tool_context: ToolContext, user_name: str, country: str) -> Dict[str, Any]:
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}

def retrieve_user_info_from_db(tool_context: ToolContext) -> Dict[str, Any]:
    user_name = tool_context.state.get("user:name")
    country = tool_context.state.get("user:country")

    return {"status": "success", "user_name": user_name, "country": country}

In [20]:
root_agent = LlmAgent(
    model=Gemini(model_name=MODEL_NAME, retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_user_info_to_db` tool. 
    * To fetch username and country when required use `retrieve_user_info_from_db` tool.
    """,
    tools=[save_user_info_to_db, retrieve_user_info_from_db],
)
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")



In [21]:
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Ayoush. I'm from India.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    "state-demo-session",
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



### Session: state-demo-session

User > Hi there, how are you doing today? What is my name?


gemini-2.5-flash-lite > I don't know your name. Could you please tell me what it is?

User > My name is Ayoush. I'm from India.


gemini-2.5-flash-lite > Nice to meet you, Ayoush from India! I've saved your name and country. Is there anything else I can help you with today?

User > What is my name? Which country am I from?


gemini-2.5-flash-lite > Your name is Ayoush and you are from India.


In [22]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name="default", user_id=USER_ID, session_id="state-demo-session"
)

print("Session State Contents:")
print(session.state)
print("\n🔍 Notice the 'user:name' and 'user:country' keys storing our data!")

Session State Contents:
{'user:name': 'Ayoush', 'user:country': 'India'}

🔍 Notice the 'user:name' and 'user:country' keys storing our data!


In [23]:
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)



### Session: new-isolated-session

User > Hi there, how are you doing today? What is my name?


gemini-2.5-flash-lite > I'm doing great, thank you!




gemini-2.5-flash-lite > Your name is Ayoush.
